<table style="width: 100%; border-collapse: collapse;" border="0">
<tr>
<td><b>Created:</b> Tuesday 31 January 2017</td>
<td style="text-align: right;"><a href="https://www.github.com/rhyswhitley/fire_limitation">github.com/rhyswhitley/fire_limitation</td>
</tr>
</table>

<div>
<center>
<font face="Times">
<br>
<h1>Quantifying the uncertainity of a global fire limitation model using Bayesian inference</h1>
<h2>Part 2: Bayesian inference</h2>
<br>
<br>
<sup>1,* </sup>Douglas Kelley, 
<sup>2 </sup>Ioannis Bistinas, 
<sup>3, 4 </sup>Chantelle Burton, 
<sup>1 </sup>Tobias Marthews, 
<sup>5 </sup>Rhys Whitley
<br>
<br>
<br>
<sup>1 </sup>Centre for Ecology and Hydrology, Maclean Building, Crowmarsh Gifford, Wallingford, Oxfordshire, United Kingdom
<br>
<sup>2 </sup>Vrije Universiteit Amsterdam, Faculty of Earth and Life Sciences, Amsterdam, Netherlands
<br>
<sup>3 </sup>Met Office United Kingdom, Exeter, United Kingdom
<br>
<sup>4 </sup>Geography, University of Exeter, Exeter, United Kingdom
<br>
<sup>5 </sup>Natural Perils Pricing, Commercial & Consumer Portfolio & Pricing, Suncorp Group, Sydney, Australia
<br>
<br>
<h3>Summary</h3>
<hr>
<p> 
This notebook aims to quantify the model parameters of a global fire model (defined below). The model is driven by a number of covariates (X<sub>i=1, 2, ... M</sub>) that describe: cropland, pasture and urban area footprints; frequency of lightening ignitions, population density, net primary productivity (NPP) and <i>Alpha</i>, a proxy measure of available soil moisture in the root zone. The model attempts to predict the impact of fire through burnt area and is thus the model target (Y).
</p>
<br>
<br>
<br>
<i>Python code and calculations below</i>
<br>
</font>
</center>
<hr>
</div>

### Model description
The model considers percentage of burnt area to be the joint product of a set of conditions that modulate fire through fuel load, ignitions, moisture and supression. Each function assumes some equilibrium point that desribes the optimal conditions for fire, that may be proportionally modified through some empirical relationship. These are briefly outlined below for the sake of comprehension in this notebook, but can be referred to in more detail in the model protocol located in the docs/ folder (<a href='file:///localhost/../docs/Model_description.pdf'>model protocol</a>).

\begin{equation}
    F_{burn} = f_{fuel}\cdot f_{ignite}\cdot(1 - f_{moist})\cdot(1 - f_{supress})
\end{equation}

Each of the above 4 functions describing the fire conditions are described by a sigmoid:

\begin{equation}
    f(x_{i=1\ldots 4}) = \frac{1}{1 + \exp\{-k\space(x_i-x_0)\}}
\end{equation}

Three of the four condition functions describe an aggregation of other climate and land-use covariates; the fuel function considers only NPP and is directly modified by changes in these values. Consequently for functions $f_{i=2\ldots4}$, the parameter $x_i$ is an aggregation of independent drivers; these are now described below.  

#### Fuel load covariate
\begin{equation}
    x_{fuel} = NPP 
\end{equation}

#### Moisture covariate
\begin{equation}
    x_{moist} = \frac{\alpha + k_p*p_{atm}}{1 + k_p}
\end{equation}

#### Ignition covariate 
\begin{equation}
    x_{ignite} = F_L\cdot\min(1.0,\space 0.0408\cdot F_L^{-0.4180}) + k_p\cdot A_{pasture} + k_{d1}\cdot\rho_{population}
\end{equation}

#### Supression covariate 
\begin{equation}
    x_{supress} = A_{urban} + k_C\cdot A_{Crop} + k_{d2}\cdot\rho_{population} 
\end{equation}

This leaves 11 free parameters that need to be optimised against observations of burnt area.  

## Load libraries

In [1]:
import pymc3 as pm
import numpy as np
import theano as th
import pandas as pd